# Класифікація даних акселерометра для визначення активності

In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from scipy.stats import skew, kurtosis

In [30]:
data_idle = pd.read_csv('data/idle/idle-1.csv')
data_running = pd.read_csv('data/running/running-1.csv')
data_stairs = pd.read_csv('data/stairs/stairs-1.csv')
data_walking = pd.read_csv('data/walking/walking-1.csv')

data_idle['label'] = 'idle'
data_running['label'] = 'running'
data_stairs['label'] = 'stairs'
data_walking['label'] = 'walking'

data = pd.concat([data_idle, data_running, data_stairs, data_walking], ignore_index=True)
data.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z,label
0,1.000776,4.616021,8.576031,idle
1,0.718261,4.209007,8.446744,idle
2,-0.909797,-0.282516,9.203311,idle
3,5.099650,0.148441,8.418014,idle
4,1.762132,-0.162806,9.251195,idle


In [31]:
for label in data['label'].unique():
    activity_data = data[data['label'] == label]
    print(f"Activity: {label}, Number of records: {len(activity_data)}")

def create_time_windows(df, window_size=30, step_size=15):
    n_rows = df.shape[0]
    windows = []
    for start in range(0, n_rows, step_size):
        end = start + window_size
        if end <= n_rows:
            windows.append(df.iloc[start:end])
    return windows



def compute_time_domain_features(window):


    features = {}


    features['mean_x'] = window['accelerometer_X'].mean()


    features['mean_y'] = window['accelerometer_Y'].mean()


    features['mean_z'] = window['accelerometer_Z'].mean()
    


    features['std_x'] = window['accelerometer_X'].std()


    features['std_y'] = window['accelerometer_Y'].std()


    features['std_z'] = window['accelerometer_Z'].std()
    


    features['skew_x'] = skew(window['accelerometer_X'])


    features['skew_y'] = skew(window['accelerometer_Y'])


    features['skew_z'] = skew(window['accelerometer_Z'])
    


    features['kurt_x'] = kurtosis(window['accelerometer_X'])


    features['kurt_y'] = kurtosis(window['accelerometer_Y'])


    features['kurt_z'] = kurtosis(window['accelerometer_Z'])
    


    return pd.Series(features)




features = []
labels = []

for label in data['label'].unique():
    activity_data = data[data['label'] == label]
    windows = create_time_windows(activity_data)
    print(f"Activity: {label}, Number of windows: {len(windows)}")
    for window in windows:
        if len(window) > 0:
            features.append(compute_time_domain_features(window))
            labels.append(label)

features_df_corrected = pd.DataFrame(features)
features_df_corrected['label'] = labels

if features_df_corrected.empty:
    raise ValueError(
        "DataFrame features_df_corrected порожній. Перевірте етапи обчислення часових ознак.")
else:
    print("DataFrame успішно створено з розміром:", features_df_corrected.shape)

Activity: idle, Number of records: 30
Activity: running, Number of records: 30
Activity: stairs, Number of records: 30
Activity: walking, Number of records: 30
Activity: idle, Number of windows: 1
Activity: running, Number of windows: 1
Activity: stairs, Number of windows: 1
Activity: walking, Number of windows: 1
DataFrame успішно створено з розміром: (4, 13)


In [32]:
X = features_df_corrected.drop(columns=['label'])
y = features_df_corrected['label']

X_train, y_train = X, y

In [34]:
svm_model = SVC(kernel='rbf', random_state=42)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_train)



rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_train)

In [35]:
report_svm = classification_report(y_train, y_pred_svm)
report_rf = classification_report(y_train, y_pred_rf)

print("SVM Classification Report:")
print(report_svm)

print("\nRandom Forest Classification Report:")
print(report_rf)

SVM Classification Report:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00         1
     running       1.00      1.00      1.00         1
      stairs       1.00      1.00      1.00         1
     walking       1.00      1.00      1.00         1

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4


Random Forest Classification Report:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00         1
     running       1.00      1.00      1.00         1
      stairs       1.00      1.00      1.00         1
     walking       1.00      1.00      1.00         1

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4

